In [1]:
from kafka import KafkaConsumer
import psycopg2
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
PG_HOST=os.getenv("PG_HOST")
PG_PORT=os.getenv("PG_PORT")
PG_USER=os.getenv("PG_USER")
PG_PASSWORD=os.getenv("PG_PASSWORD")
PG_DATABASE=os.getenv("PG_DATABASE")

In [4]:
TOPIC_NAME = os.getenv("KAFKA_TOPIC1")
KAFKA_BOOTSTRAP_SERVER = ['localhost:9092']

In [5]:
consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVER,
    auto_offset_reset='earliest'
)

In [6]:
conn = psycopg2.connect(
    host=PG_HOST,
    port=PG_PORT,
    user=PG_USER,
    password=PG_PASSWORD,
    database=PG_DATABASE
)
cur = conn.cursor()

In [7]:
insert_query = """
INSERT INTO metrics (id, cpu, mem, disk)
VALUES (%s, %s, %s, %s)
"""

In [ ]:
for message in consumer:
    data = message.value

    value = message.value.decode('utf-8')
    numbers = tuple([int(item.split(":")[1].strip()) for item in value.split(",")])
 
    # Insert data into PostgreSQL table
    cur.execute(insert_query,numbers)
    conn.commit()

    print(f"Received message:value = {numbers}")


Received message:value = (4, 38, 88, 100)
Received message:value = (2, 53, 46, 54)
Received message:value = (3, 86, 61, 60)
Received message:value = (0, 20, 94, 13)
Received message:value = (8, 63, 21, 81)
Received message:value = (5, 38, 15, 2)
Received message:value = (1, 43, 67, 8)
Received message:value = (7, 87, 67, 29)
Received message:value = (6, 11, 24, 16)
Received message:value = (9, 92, 37, 45)
Received message:value = (6, 25, 79, 95)
Received message:value = (1, 60, 11, 47)
Received message:value = (8, 66, 98, 38)
Received message:value = (2, 94, 72, 100)
Received message:value = (5, 94, 72, 4)
Received message:value = (7, 78, 70, 93)
Received message:value = (4, 37, 24, 92)
Received message:value = (0, 39, 71, 47)
Received message:value = (3, 57, 58, 21)
Received message:value = (0, 24, 59, 21)
Received message:value = (1, 18, 16, 84)
Received message:value = (7, 78, 61, 30)
Received message:value = (5, 80, 61, 71)
Received message:value = (2, 50, 26, 39)
Received message:

In [ ]:
# Close connections
consumer.close()
cur.close()
conn.close()